# K-Means vs. KNN: Comprehensive Comparison

This notebook demonstrates the key differences between K-Means Clustering and K-Nearest Neighbors (KNN) through practical examples.

## Overview
- **K-Means**: Unsupervised clustering algorithm
- **KNN**: Supervised learning algorithm for classification and regression

In [ ]:
# Install required packages if needed
# !pip install numpy pandas matplotlib seaborn scikit-learn

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.datasets import make_classification, make_regression, make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, silhouette_score
from sklearn.impute import KNNImputer
import time
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seed for reproducibility
np.random.seed(42)

---
## 1. K-Means: Unsupervised Clustering

K-Means groups data into clusters without using labels. It's useful for discovering patterns in unlabeled data.

In [ ]:
# Generate synthetic data for clustering
X_clusters, y_true = make_blobs(n_samples=300, centers=4, n_features=2, 
                                 cluster_std=0.60, random_state=42)

# Apply K-Means
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
y_kmeans = kmeans.fit_predict(X_clusters)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Original data (without labels - unsupervised scenario)
axes[0].scatter(X_clusters[:, 0], X_clusters[:, 1], c='gray', alpha=0.5, s=50)
axes[0].set_title('Original Data (Unlabeled)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')

# K-Means clustering result
axes[1].scatter(X_clusters[:, 0], X_clusters[:, 1], c=y_kmeans, cmap='viridis', s=50)
axes[1].scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], 
                c='red', marker='X', s=300, edgecolors='black', linewidths=2, label='Centroids')
axes[1].set_title('K-Means Clustering (k=4)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Silhouette Score: {silhouette_score(X_clusters, y_kmeans):.3f}")
print("\n✓ K-Means discovers patterns in unlabeled data!")

---
## 2. KNN: Classification (Supervised Learning)

KNN predicts the class of a new point based on the classes of its k nearest neighbors. It requires labeled training data.

In [ ]:
# Generate classification dataset
X_class, y_class = make_classification(n_samples=400, n_features=2, n_redundant=0, 
                                        n_informative=2, n_clusters_per_class=1,
                                        random_state=42, flip_y=0.05)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_class, y_class, test_size=0.3, random_state=42)

# Train KNN classifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train, y_train)

# Predictions
y_pred = knn_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Create mesh for decision boundary
h = 0.02
x_min, x_max = X_class[:, 0].min() - 1, X_class[:, 0].max() + 1
y_min, y_max = X_class[:, 1].min() - 1, X_class[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = knn_clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training data
axes[0].scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap='coolwarm', 
                edgecolors='black', s=50, alpha=0.7)
axes[0].set_title('Training Data (Labeled)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')

# Decision boundary and test predictions
axes[1].contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
axes[1].scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap='coolwarm', 
                edgecolors='black', s=50, alpha=0.8, label='True Labels')
axes[1].set_title(f'KNN Classification (k=5) - Accuracy: {accuracy:.3f}', 
                  fontsize=14, fontweight='bold')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Test Accuracy: {accuracy:.3f}")
print("\n✓ KNN uses labeled data to predict new samples!")

---
## 3. KNN: Regression

KNN can also predict continuous values by averaging the target values of k nearest neighbors.

In [ ]:
# Generate regression dataset
X_reg = np.sort(5 * np.random.rand(200, 1), axis=0)
y_reg = np.sin(X_reg).ravel() + np.random.normal(0, 0.1, X_reg.shape[0])

# Split data
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=42
)

# Train KNN regressor with different k values
k_values = [1, 5, 20]
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

X_test_sorted = np.linspace(0, 5, 300)[:, np.newaxis]

for idx, k in enumerate(k_values):
    knn_reg = KNeighborsRegressor(n_neighbors=k)
    knn_reg.fit(X_train_reg, y_train_reg)
    y_pred_reg = knn_reg.predict(X_test_sorted)
    
    # Calculate MSE on test set
    y_test_pred = knn_reg.predict(X_test_reg)
    mse = mean_squared_error(y_test_reg, y_test_pred)
    
    axes[idx].scatter(X_train_reg, y_train_reg, c='blue', s=20, alpha=0.5, label='Training Data')
    axes[idx].plot(X_test_sorted, y_pred_reg, c='red', linewidth=2, label='KNN Prediction')
    axes[idx].set_title(f'KNN Regression (k={k})\nMSE: {mse:.3f}', 
                       fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('X')
    axes[idx].set_ylabel('y')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ KNN regression averages k nearest neighbors' values!")
print("Note: Smaller k = more flexible (may overfit), Larger k = smoother predictions")

---
## 4. Missing Value Imputation

Both K-Means and KNN can be used for imputing missing values, but KNN is more commonly used and often more effective.

In [ ]:
# Create dataset with missing values
np.random.seed(42)
n_samples = 200
X_complete = np.random.randn(n_samples, 3) * 10 + 50
X_complete[:, 1] = X_complete[:, 0] * 0.8 + np.random.randn(n_samples) * 5  # Create correlation

# Introduce missing values (15% of data)
X_missing = X_complete.copy()
missing_mask = np.random.random(X_missing.shape) < 0.15
X_missing[missing_mask] = np.nan

print(f"Total values: {X_missing.size}")
print(f"Missing values: {np.isnan(X_missing).sum()} ({np.isnan(X_missing).sum()/X_missing.size*100:.1f}%)")
print()

# Method 1: KNN Imputation
knn_imputer = KNNImputer(n_neighbors=5)
X_knn_imputed = knn_imputer.fit_transform(X_missing)

# Method 2: K-Means based imputation (manual implementation)
def kmeans_imputation(X, n_clusters=5):
    X_imputed = X.copy()
    
    # First pass: fill with column means for initial clustering
    col_means = np.nanmean(X, axis=0)
    for i in range(X.shape[1]):
        X_imputed[np.isnan(X_imputed[:, i]), i] = col_means[i]
    
    # Cluster the data
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_imputed)
    
    # Impute based on cluster means
    X_final = X.copy()
    for cluster_id in range(n_clusters):
        cluster_mask = clusters == cluster_id
        cluster_means = np.nanmean(X[cluster_mask], axis=0)
        
        for i in range(X.shape[1]):
            missing_in_cluster = np.isnan(X[:, i]) & cluster_mask
            X_final[missing_in_cluster, i] = cluster_means[i]
    
    return X_final

X_kmeans_imputed = kmeans_imputation(X_missing, n_clusters=5)

# Calculate imputation errors
missing_indices = np.where(missing_mask)
knn_error = np.mean((X_complete[missing_indices] - X_knn_imputed[missing_indices])**2)
kmeans_error = np.mean((X_complete[missing_indices] - X_kmeans_imputed[missing_indices])**2)

print(f"KNN Imputation MSE: {knn_error:.3f}")
print(f"K-Means Imputation MSE: {kmeans_error:.3f}")
print()

# Visualize imputation quality for first two features
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Original complete data
axes[0].scatter(X_complete[:, 0], X_complete[:, 1], alpha=0.5, s=30)
axes[0].set_title('Original Complete Data', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')

# KNN imputation
axes[1].scatter(X_complete[:, 0], X_complete[:, 1], alpha=0.3, s=20, label='Original', color='gray')
imputed_mask = missing_mask[:, 0] | missing_mask[:, 1]
axes[1].scatter(X_knn_imputed[imputed_mask, 0], X_knn_imputed[imputed_mask, 1], 
                alpha=0.8, s=50, label='KNN Imputed', color='red', marker='^')
axes[1].set_title(f'KNN Imputation (MSE: {knn_error:.3f})', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
axes[1].legend()

# K-Means imputation
axes[2].scatter(X_complete[:, 0], X_complete[:, 1], alpha=0.3, s=20, label='Original', color='gray')
axes[2].scatter(X_kmeans_imputed[imputed_mask, 0], X_kmeans_imputed[imputed_mask, 1], 
                alpha=0.8, s=50, label='K-Means Imputed', color='blue', marker='s')
axes[2].set_title(f'K-Means Imputation (MSE: {kmeans_error:.3f})', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Feature 1')
axes[2].set_ylabel('Feature 2')
axes[2].legend()

plt.tight_layout()
plt.show()

print("\n✓ KNN typically provides better imputation by using local neighborhood information!")

---
## 5. Runtime Comparison

Let's compare the computational complexity of both algorithms with increasing dataset sizes.

In [ ]:
# Test different dataset sizes
sample_sizes = [100, 500, 1000, 2000, 5000]
n_features = 10
k = 5

results = {
    'n_samples': [],
    'kmeans_fit': [],
    'kmeans_predict': [],
    'knn_fit': [],
    'knn_predict': []
}

print("Running runtime benchmarks...\n")

for n in sample_sizes:
    print(f"Testing with {n} samples...")
    
    # Generate data
    X, y = make_classification(n_samples=n, n_features=n_features, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    results['n_samples'].append(n)
    
    # K-Means timing
    start = time.time()
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_train)
    results['kmeans_fit'].append(time.time() - start)
    
    start = time.time()
    _ = kmeans.predict(X_test)
    results['kmeans_predict'].append(time.time() - start)
    
    # KNN timing
    start = time.time()
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    results['knn_fit'].append(time.time() - start)
    
    start = time.time()
    _ = knn.predict(X_test)
    results['knn_predict'].append(time.time() - start)

# Convert to DataFrame for easier plotting
df_results = pd.DataFrame(results)

# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training time
axes[0].plot(df_results['n_samples'], df_results['kmeans_fit'], 
             marker='o', linewidth=2, markersize=8, label='K-Means Fit')
axes[0].plot(df_results['n_samples'], df_results['knn_fit'], 
             marker='s', linewidth=2, markersize=8, label='KNN Fit')
axes[0].set_xlabel('Number of Samples', fontsize=12)
axes[0].set_ylabel('Time (seconds)', fontsize=12)
axes[0].set_title('Training Time Comparison', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Prediction time
axes[1].plot(df_results['n_samples'], df_results['kmeans_predict'], 
             marker='o', linewidth=2, markersize=8, label='K-Means Predict')
axes[1].plot(df_results['n_samples'], df_results['knn_predict'], 
             marker='s', linewidth=2, markersize=8, label='KNN Predict')
axes[1].set_xlabel('Number of Samples', fontsize=12)
axes[1].set_ylabel('Time (seconds)', fontsize=12)
axes[1].set_title('Prediction Time Comparison', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "="*70)
print("RUNTIME SUMMARY")
print("="*70)
print(df_results.to_string(index=False))
print("="*70)

print("\n📊 Key Observations:")
print("  • K-Means: Slow training (iterative), FAST prediction (just find nearest centroid)")
print("  • KNN: INSTANT training (lazy learner), slow prediction (searches all training data)")
print("  • KNN prediction time grows with training set size (linear complexity)")
print("  • K-Means prediction time stays constant (independent of training size)")

---
## 6. Summary: When to Use Which?

### Use K-Means when:
- ✓ You have **unlabeled data** and want to discover patterns
- ✓ You need **fast predictions** after training
- ✓ You want to do **customer segmentation**, image compression, or feature engineering
- ✓ You can define the number of clusters upfront

### Use KNN when:
- ✓ You have **labeled data** and want to make predictions
- ✓ You need both **classification and regression** capabilities
- ✓ You want to **impute missing values** effectively
- ✓ You need a simple, interpretable model
- ✓ Training speed is more important than prediction speed

### Key Differences:

| Aspect | K-Means | KNN |
|--------|---------|-----|
| **Learning Type** | Unsupervised | Supervised |
| **Training Time** | Slow (iterative) | Instant (lazy) |
| **Prediction Time** | Fast (O(k)) | Slow (O(n)) |
| **Use Cases** | Clustering, segmentation | Classification, regression, imputation |
| **Requires Labels** | No | Yes |
| **Output** | Cluster assignments | Class labels or continuous values |

In [ ]:
# Final comparison visualization
fig, ax = plt.subplots(figsize=(12, 6))

categories = ['Unsupervised\nLearning', 'Classification', 'Regression', 
              'Fast\nPrediction', 'Missing Value\nImputation']
kmeans_scores = [1.0, 0.1, 0.0, 1.0, 0.6]  # Capability scores
knn_scores = [0.0, 1.0, 1.0, 0.3, 1.0]

x = np.arange(len(categories))
width = 0.35

bars1 = ax.bar(x - width/2, kmeans_scores, width, label='K-Means', 
               color='steelblue', alpha=0.8)
bars2 = ax.bar(x + width/2, knn_scores, width, label='KNN', 
               color='coral', alpha=0.8)

ax.set_ylabel('Capability Score', fontsize=12)
ax.set_title('K-Means vs. KNN: Capability Comparison', fontsize=16, fontweight='bold', pad=20)
ax.set_xticks(x)
ax.set_xticklabels(categories, fontsize=11)
ax.legend(fontsize=12)
ax.set_ylim(0, 1.2)
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        if height > 0.1:
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.1f}',
                   ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🎯 Choose the right tool for your specific problem!")